<a href="https://colab.research.google.com/github/MilciadesGonzalez/TradeLaVirginia/blob/main/Parametros.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Crudo Dorinka

In [ ]:
crudoDorinka = pd.read_excel('/content/drive/Shareddrives/Trade Mktg/Trade/1.5 - Parametros/0 - Parametros con Python/1 - Crudo/kx1l7wl_801682041.xlsx', 
                             skiprows=26, names=['nro tienda','Tienda','UPC','Item Nbr','Item Flags','Modular Items Flag','Range 1 Curr Traited Store/Item Comb.',
                                                 'Range 1 Venta u','Range 1 Stock tienda','Range 1 Stock en tansito','Range 1 Stock CD','Range 1 Stock on order',
                                                 'Range 1 Store Weeks Supply','Range 1 Carga gondola','Range 2 Curr Traited Store/Item Comb.','Range 2 Venta u',
                                                 'Range 2 Stock tienda','Range 2 Stock en tansito','Range 2 Stock CD','Range 2 Stock on order','Range 2 Store Weeks Supply',
                                                 'Range 2 Carga gondola'])

Datos externo: Boca - Paletizado - Jerarquia

In [ ]:
bocasDorinka = pd.read_excel('/content/drive/Shareddrives/Trade Mktg/Trade/1.5 - Parametros/0 - Parametros con Python/1 - DatosExternos/Bocas.xlsx', sheet_name='Dorinka')
bocasCencosud = pd.read_excel('/content/drive/Shareddrives/Trade Mktg/Trade/1.5 - Parametros/0 - Parametros con Python/1 - DatosExternos/Bocas.xlsx', sheet_name='Cencosud')
bocasLaAnonima = pd.read_excel('/content/drive/Shareddrives/Trade Mktg/Trade/1.5 - Parametros/0 - Parametros con Python/1 - DatosExternos/Bocas.xlsx', sheet_name='La Anonima')
bocasInc = pd.read_excel('/content/drive/Shareddrives/Trade Mktg/Trade/1.5 - Parametros/0 - Parametros con Python/1 - DatosExternos/Bocas.xlsx', sheet_name='INC')
paletizado = pd.read_excel('/content/drive/Shareddrives/Trade Mktg/Trade/1.5 - Parametros/0 - Parametros con Python/1 - DatosExternos/Paletizado.xlsx')
jerarquia = pd.read_excel('/content/drive/Shareddrives/Trade Mktg/Trade/1.5 - Parametros/0 - Parametros con Python/1 - DatosExternos/1 - Master Jerarquia de Materiales.xlsx')

Union de bases

In [ ]:
dorinka = crudoDorinka.merge(bocasDorinka, on='nro tienda')
dorinka = dorinka.merge(jerarquia, on='UPC')
dorinka = dorinka.merge(paletizado, on='UPC')

Limpieza y modificación

In [ ]:
dorinka['Range 1 Curr Traited Store/Item Comb.'] = np.select([dorinka['Range 1 Curr Traited Store/Item Comb.']>0,
                                                              dorinka['Range 1 Curr Traited Store/Item Comb.']==0],('M', 'I'))
dorinka['Modular Items Flag'] = np.select([dorinka['Modular Items Flag']=='M',dorinka['Modular Items Flag']==""],(1, 0))
dorinka = dorinka.rename(columns={"Range 1 Curr Traited Store/Item Comb.":"Status","Modular Items Flag":"Modular","UPC":"EAN","Item Nbr":"CodigoArticulo",
                                  "Texto breve de material":"DescripcionProducto","Range 1 Stock tienda":"StockTienda","Range 1 Stock en tansito":"StockTransito",
                                  "Range 1 Stock CD":"StockCD","Range 1 Carga gondola":"CargaGondola","Range 1 Venta u":"VentaSemanal"})
dorinka['CoberturaDias'] = np.select([dorinka['VentaSemanal']>0,dorinka['VentaSemanal']<=0],[dorinka['StockTienda']/dorinka['VentaSemanal'],0])
dorinka['Observaciones'] = np.select([dorinka['StockTienda']<0,((dorinka['StockTienda']>0) & (dorinka['VentaSemanal']==0))],['STOCK NEGATIVO','SIN VENTAS UNA SEMANA'])
dorinka['CargaMinima'] = ""
dorinka['Fecha'] = ''

Orden de columnas

In [ ]:
dorinka = dorinka.drop(["nro tienda", "Tienda", "Item Flags", "Range 1 Stock on order", "Range 1 Store Weeks Supply", "Range 2 Curr Traited Store/Item Comb.",
                       "Range 2 Venta u", "Range 2 Stock tienda", "Range 2 Stock en tansito", "Range 2 Stock CD", "Range 2 Stock on order", "Range 2 Store Weeks Supply",
                       "Range 2 Carga gondola"], axis=1)
dorinka = dorinka[['Codigo','Status','Modular','EAN','CodigoArticulo','DescripcionProducto','VentaSemanal','CoberturaDias','StockTienda','StockTransito','StockCD',
                   'CargaGondola','CargaMinima','Observaciones','Fecha','Jerarquia']]
dorinka['Observaciones'] = np.where((dorinka.Observaciones=='0'), ' ', dorinka.Observaciones)

In [ ]:
dorinka = dorinka.rename(columns={'Codigo':'CodigoCliente','Status':'StatusDelArticulo','Modular':'ItemModulado','CodigoArticulo':'CodigoArticuloCliente','DescripcionProducto':'DescripcionLV',
                                  'VentaSemanal':'VentaQsemanal','Fecha':'FechaImportacion'})

Exportar resultado

In [ ]:
# dorinka.to_excel('ParDorinka.xlsx')
# !cp ParDorinka.xlsx "/content/drive/Shareddrives/Trade Mktg/Trade/1.5 - Parametros/0 - Parametros con Python/"

Crudo La Anonima

In [ ]:
crudoVentaLaAnonima = pd.read_excel('/content/drive/Shareddrives/Trade Mktg/Trade/1.5 - Parametros/0 - Parametros con Python/1 - Crudo/Dossier - Venta x Dia.xlsx',
                                    skiprows=3, names=['Proveedor_SAP_Portal','Proveedor - Division','Region','Zona','Sucursal','Sector','Familia','SubFamilia','Articulo ID',
                                                       'Articulo DESC','Articulo Marca','Articulo Contenido','Articulo Unidad de Medida','Scancode CD','ind','u1','v1','u2','v2',
                                                       'u3','v3','u4','v4','u5','v5','u6','v6','u7','v7'])
crudoStockLaAnonima = pd.read_excel('/content/drive/Shareddrives/Trade Mktg/Trade/1.5 - Parametros/0 - Parametros con Python/1 - Crudo/Dossier - Stock x Dia.xlsx',
                                    skiprows=3, names=['Proveedor - Division','Region','Zona','Sucursal','Sector','Familia','SubFamilia','Articulo ID','Articulo DESC',
                                                       'Articulo Marca','Articulo Contenido','Articulo Unidad de Medida','ind','st1','st2','st3','st4','st5','st6','st7'])

Modificaciones La Anonima

In [ ]:
crudoVentaLaAnonima = crudoVentaLaAnonima.replace(np.nan,0)
crudoVentaLaAnonima['VentaSemanal'] = crudoVentaLaAnonima['u1']+crudoVentaLaAnonima['u2']+crudoVentaLaAnonima['u3']+crudoVentaLaAnonima['u4']+crudoVentaLaAnonima['u5']+crudoVentaLaAnonima['u6']+crudoVentaLaAnonima['u7']
crudoVentaLaAnonima['Concat'] = crudoVentaLaAnonima['Sucursal'].astype(str)+'-'+crudoVentaLaAnonima['Articulo ID'].astype(str)
crudoStockLaAnonima = crudoStockLaAnonima.replace(np.nan,0)
crudoStockLaAnonima['Stock'] = crudoStockLaAnonima['st1']+crudoStockLaAnonima['st2']+crudoStockLaAnonima['st3']+crudoStockLaAnonima['st4']+crudoStockLaAnonima['st5']+crudoStockLaAnonima['st6']+crudoStockLaAnonima['st7']
crudoStockLaAnonima['Concat'] = crudoStockLaAnonima['Sucursal'].astype(str)+'-'+crudoStockLaAnonima['Articulo ID'].astype(str)

Union y Limpieza

In [ ]:
from numpy.core.numeric import outer
crudoVentaLaAnonima = pd.merge(crudoVentaLaAnonima,bocasLaAnonima, on='Sucursal')
crudoStockLaAnonima = pd.merge(crudoStockLaAnonima,bocasLaAnonima, on='Sucursal')
auxVta = crudoVentaLaAnonima.copy()
crudoVentaLaAnonima = crudoVentaLaAnonima.drop(['Proveedor_SAP_Portal','Proveedor - Division','Sucursal','Scancode CD','Region','Zona','Sector','Familia','SubFamilia',
                                                'Articulo DESC','Articulo Marca','Articulo Contenido','Articulo Unidad de Medida','ind','u1','v1','u2','v2',
                                                'u3','v3','u4','v4','u5','v5','u6','v6','u7','v7'], axis=1)
auxVta = auxVta.drop(['Proveedor_SAP_Portal','Proveedor - Division','Sucursal','VentaSemanal','Concat','Codigo','Region','Zona','Sector','Familia','SubFamilia',
                                                'Articulo DESC','Articulo Marca','Articulo Contenido','Articulo Unidad de Medida','ind','u1','v1','u2','v2',
                                                'u3','v3','u4','v4','u5','v5','u6','v6','u7','v7'], axis=1)
auxVta = auxVta.drop_duplicates()
crudoStockLaAnonima = crudoStockLaAnonima.drop(['Proveedor - Division','Region','Zona','Sector','Sucursal','Familia','SubFamilia','Articulo DESC',
                                                'Articulo Marca','Articulo Contenido','Articulo Unidad de Medida','ind','st1','st2','st3','st4','st5','st6','st7'], axis=1)
ventaLaAnonima = pd.merge(crudoVentaLaAnonima,crudoStockLaAnonima, on='Concat', how='left')
sinVentaLaAnonima = pd.merge(crudoVentaLaAnonima,crudoStockLaAnonima, on='Concat', how='right')
sinVentaLaAnonima = sinVentaLaAnonima.loc[sinVentaLaAnonima['VentaSemanal'].isnull()]
laAnonima = pd.concat([ventaLaAnonima,sinVentaLaAnonima],ignore_index=True)
laAnonima = laAnonima.drop(['Articulo ID_x','Articulo ID_x','Concat','Codigo_x'], axis=1)
laAnonima = pd.merge(laAnonima,auxVta,left_on='Articulo ID_y',right_on='Articulo ID',how='left')
laAnonima = laAnonima.drop(['Articulo ID'], axis=1)
laAnonima = pd.merge(laAnonima,paletizado,left_on='Scancode CD',right_on='EAN')
laAnonima = laAnonima.drop(['UPC', 'Scancode CD'], axis=1)
laAnonima['Cobertura'] = np.select([laAnonima['VentaSemanal']>0,laAnonima['VentaSemanal']<=0],[laAnonima['Stock']/laAnonima['VentaSemanal'],0])
laAnonima['Observaciones'] = np.select([laAnonima['Stock']<0,((laAnonima['Stock']>0) & (laAnonima['VentaSemanal']==0))],['STOCK NEGATIVO','SIN VENTAS UNA SEMANA'])
laAnonima = pd.merge(laAnonima,jerarquia,on='EAN')
laAnonima = laAnonima.drop(['UPC'], axis=1)

Orden de columnas

In [ ]:
laAnonima['StatusDelArticulo'] = ""
laAnonima['ItemModulado'] = ""
laAnonima['StockTransito'] = ""
laAnonima['StockCD'] = ""
laAnonima['CargaGondola'] = ""
laAnonima['CargaMinima'] = ""
laAnonima['FechaImportacion'] = ''
laAnonima = laAnonima[['Codigo_y','StatusDelArticulo','ItemModulado','EAN','Articulo ID_y','Texto breve de material','VentaSemanal','Cobertura','Stock',
                       'StockTransito','StockCD','CargaGondola','CargaMinima','Observaciones','FechaImportacion','Jerarquia']]
laAnonima['Observaciones'] = np.where((laAnonima.Observaciones=='0'), ' ', laAnonima.Observaciones)
laAnonima = laAnonima[laAnonima['VentaSemanal'].notna()]
laAnonima = laAnonima.rename(columns={'Codigo_y':'CodigoCliente','Articulo ID_y':'CodigoArticuloCliente','Texto breve de material':'DescripcionLV','VentaSemanal':'VentaQsemanal',
                                  'Cobertura':'CoberturaDias','Stock':'StockTienda'})

Exportar resultado

In [ ]:
# laAnonima.to_excel('laAnonima.xlsx')
# !cp laAnonima.xlsx "/content/drive/Shareddrives/Trade Mktg/Trade/1.5 - Parametros/0 - Parametros con Python/"

Crudo Cencosud

In [ ]:
crudoCencoReg1 = pd.read_csv('/content/drive/Shareddrives/Trade Mktg/Trade/1.5 - Parametros/0 - Parametros con Python/1 - Crudo/NivServ_AM_Reg1.csv',skiprows=4,index_col=0, encoding='latin-1',sep=';',
                             decimal=",", names=['Region','Cadena','Local','Seccion','Familia','Item','Descripcion','Codigo Interno','% NS','% NS1','% NS2','Stock','Pend Entrega','Dias Stock',
                                    'Dias Obj','Cob Obj Un','Vta Prom DiariaNeta','Pco Uni','Promo','UxB','Fecha Ult Vta','Dias Sin Vta con Stk','Forma Entrega','Vigencia',
                                    'Rol Local','Costo','Carga Gondola','CD Abastece','vacio'])
crudoCencoReg3 = pd.read_csv('/content/drive/Shareddrives/Trade Mktg/Trade/1.5 - Parametros/0 - Parametros con Python/1 - Crudo/NivServ_AM_Reg3.csv',skiprows=4,index_col=0, encoding='latin-1',sep=';',
                             decimal=",", names=['Region','Cadena','Local','Seccion','Familia','Item','Descripcion','Codigo Interno','% NS','% NS1','% NS2','Stock','Pend Entrega','Dias Stock',
                                    'Dias Obj','Cob Obj Un','Vta Prom DiariaNeta','Pco Uni','Promo','UxB','Fecha Ult Vta','Dias Sin Vta con Stk','Forma Entrega','Vigencia',
                                    'Rol Local','Costo','Carga Gondola','CD Abastece','vacio'])
crudoCencoReg5 = pd.read_csv('/content/drive/Shareddrives/Trade Mktg/Trade/1.5 - Parametros/0 - Parametros con Python/1 - Crudo/NivServ_AM_Reg5.csv',skiprows=4,index_col=0, encoding='latin-1',sep=';',
                             decimal=",", names=['Region','Cadena','Local','Seccion','Familia','Item','Descripcion','Codigo Interno','% NS','% NS1','% NS2','Stock','Pend Entrega','Dias Stock',
                                    'Dias Obj','Cob Obj Un','Vta Prom DiariaNeta','Pco Uni','Promo','UxB','Fecha Ult Vta','Dias Sin Vta con Stk','Forma Entrega','Vigencia',
                                    'Rol Local','Costo','Carga Gondola','CD Abastece','vacio'])

Filtrado de stock de CDS

In [ ]:
filtroCDReg1 = crudoCencoReg1['Local']=='217 - Centro Distribución Ezei'
cdReg1 = crudoCencoReg1[filtroCDReg1]
cdReg1 = cdReg1.drop(['Cadena','Local','Seccion','Familia','Descripcion','Codigo Interno','% NS','% NS1','% NS2','Pend Entrega','Dias Stock',
                      'Dias Obj','Cob Obj Un','Vta Prom DiariaNeta','Pco Uni','Promo','UxB','Fecha Ult Vta','Dias Sin Vta con Stk','Forma Entrega','Vigencia',
                      'Rol Local','Costo','Carga Gondola','CD Abastece','vacio'], axis=1)
filtroCDReg3 = crudoCencoReg3['Local']=='779 - Centro Distribución Córd'
cdReg3= crudoCencoReg3[filtroCDReg3]
cdReg3= cdReg3.drop(['Cadena','Local','Seccion','Familia','Descripcion','Codigo Interno','% NS','% NS1','% NS2','Pend Entrega','Dias Stock',
                      'Dias Obj','Cob Obj Un','Vta Prom DiariaNeta','Pco Uni','Promo','UxB','Fecha Ult Vta','Dias Sin Vta con Stk','Forma Entrega','Vigencia',
                      'Rol Local','Costo','Carga Gondola','CD Abastece','vacio'], axis=1)
filtroCDReg5 = crudoCencoReg5['Local']=='221 - Centro Distribución Cuyo'
cdReg5 = crudoCencoReg5[filtroCDReg5]
cdReg5 = cdReg5.drop(['Cadena','Local','Seccion','Familia','Descripcion','Codigo Interno','% NS','% NS1','% NS2','Pend Entrega','Dias Stock',
                      'Dias Obj','Cob Obj Un','Vta Prom DiariaNeta','Pco Uni','Promo','UxB','Fecha Ult Vta','Dias Sin Vta con Stk','Forma Entrega','Vigencia',
                      'Rol Local','Costo','Carga Gondola','CD Abastece','vacio'], axis=1)

Union de bases

In [ ]:
crudoCencoReg1 = pd.merge(crudoCencoReg1,cdReg1,on='Item',how='left')
crudoCencoReg3 = pd.merge(crudoCencoReg3,cdReg3,on='Item',how='left')
crudoCencoReg5 = pd.merge(crudoCencoReg5,cdReg5,on='Item',how='left')
crudoCencoReg1['Stock CD'] = crudoCencoReg1['Stock_y']
crudoCencoReg3['Stock CD'] = crudoCencoReg3['Stock_y']
crudoCencoReg5['Stock CD'] = crudoCencoReg5['Stock_y']
cencosud = pd.concat([crudoCencoReg1,crudoCencoReg3,crudoCencoReg5], ignore_index=True)

Transformacion

In [ ]:
cencosud[['CodLocal','Local','Local1']] = cencosud.Local.str.split('-',expand=True)
cencosud['CodLocal'] = cencosud['CodLocal'].astype(int)
cencosud = pd.merge(cencosud,bocasCencosud, on='CodLocal')
cencosud[['EAN1','EAN']] = cencosud.Item.str.split("'",expand=True)
cencosud['EAN'] = cencosud['EAN'].astype(int)
cencosud = pd.merge(cencosud,paletizado, on='EAN')
cencosud['Vigencia'] = cencosud['Vigencia'].replace('V','A')
cencosud['Rol Local'] = cencosud['Rol Local'].replace('SI',1)
cencosud['Rol Local'] = cencosud['Rol Local'].replace('NO',0)
cencosud['ventaSemanal'] = cencosud['Vta Prom DiariaNeta']*7
cencosud['CoberturaDias'] = np.select([cencosud['ventaSemanal']>0,cencosud['ventaSemanal']<=0],[(cencosud['Stock_x']/cencosud['ventaSemanal']),0])
cencosud['Observaciones'] = np.select([cencosud['Vigencia']=='A',cencosud['Vigencia']!='A'],[(np.select([cencosud['Stock_x']<0,((cencosud['Stock_x']>0) & 
                                                                                                                                ((cencosud['ventaSemanal']==0) & (cencosud['Dias Sin Vta con Stk']>9)))],
                                                                                                        ['STOCK NEGATIVO','SiN VENTAS CON STOCK'])),""])
cencosud['Observaciones'] = np.where((cencosud.Observaciones=='0'), ' ', cencosud.Observaciones)
cencosud['Observaciones'] = np.where((cencosud.Observaciones=='SiN VENTAS CON STOCK'), ((cencosud['Dias Sin Vta con Stk'].astype(str))+' DIAS SiN VENTAS CON STOCK'), cencosud.Observaciones)
cencosud = cencosud.drop(['Cadena','Local','Seccion','Familia','Item','Descripcion','% NS','% NS1','% NS2','Dias Stock','Dias Obj','Cob Obj Un','Vta Prom DiariaNeta','Pco Uni','Promo','UxB',
                          'Fecha Ult Vta','Dias Sin Vta con Stk','Forma Entrega','Costo','CD Abastece','vacio','Stock_y','CodLocal','Local1','EAN1','UPC'], axis=1)
cencosud['CargaMinima'] = ""
cencosud['FechaImportacion'] = ''
cencosud = pd.merge(cencosud,jerarquia,on='EAN')
cencosud = cencosud.drop(['UPC'], axis=1)

Orden de columnas

In [ ]:
cencosud = cencosud[['Codigo','Vigencia','Rol Local','EAN','Codigo Interno','Texto breve de material','ventaSemanal','CoberturaDias','Stock_x','Pend Entrega','Stock CD','Carga Gondola','CargaMinima','Observaciones','FechaImportacion','Jerarquia']]

In [ ]:
cencosud = cencosud.rename(columns={'Codigo':'CodigoCliente','Vigencia':'StatusDelArticulo','Rol Local':'ItemModulado','Codigo Interno':'CodigoArticuloCliente', 'Texto breve de material':'DescripcionLV',
                                    'ventaSemanal':'VentaQsemanal','Stock_x':'StockTienda','Pend Entrega':'StockTransito','Stock CD':'StockCD','Carga Gondola':'CargaGondola'})

Exportar resultado

In [ ]:
# cencosud.to_excel('cencosud.xlsx')
# !cp cencosud.xlsx "/content/drive/Shareddrives/Trade Mktg/Trade/1.5 - Parametros/0 - Parametros con Python/"

Crudo INC

In [ ]:
crudoINC = pd.read_excel('/content/drive/Shareddrives/Trade Mktg/Trade/1.5 - Parametros/0 - Parametros con Python/1 - Crudo/20190219 Parametros Carrefour v1.xlsx')

Transformaciones INC

In [ ]:
crudoINC['CoberturaDias'] = np.select([crudoINC['VentaQsemanal']!=0,crudoINC['VentaQsemanal']==0],[((crudoINC['StockTienda']/crudoINC['VentaQsemanal'])/7),0])
crudoINC['Observaciones'] = np.select([crudoINC['StockTienda']<0,((crudoINC['StockTienda']>0) & (crudoINC['VentaQsemanal']==0))],['STOCK NEGATIVO','SIN VENTAS UN MES'])
crudoINC['Observaciones'] = np.where((crudoINC.Observaciones=='0'), ' ', crudoINC.Observaciones)
crudoINC['StatusDelArticulo'] = ''
crudoINC['ItemModulado'] = ''
crudoINC['CodigoArticuloCliente'] = ''
crudoINC['CargaGondola'] = ''
crudoINC['CargaMinima'] = ''
crudoINC['FechaImportacion'] = ''

Union de bases

In [ ]:
inc = crudoINC.merge(bocasInc, on='Local')
inc = inc.merge(jerarquia, on='EAN')
inc = inc.merge(paletizado, on='EAN')

Limpieza

In [ ]:
inc = inc.drop(['Aux2','Local','Vta Media','UPC_x','UPC_y'], axis=1)
inc = inc.rename(columns={'Texto breve de material':'DescripcionLV'})

Orden de columnas

In [ ]:
inc = inc[['CodigoCliente','StatusDelArticulo','ItemModulado','EAN','CodigoArticuloCliente','DescripcionLV','VentaQsemanal','CoberturaDias',
           'StockTienda','StockTransito','StockCD','CargaGondola','CargaMinima','Observaciones','FechaImportacion','Jerarquia']]

Exportar resultado

In [ ]:
# inc.to_excel('inc.xlsx')
# !cp inc.xlsx "/content/drive/Shareddrives/Trade Mktg/Trade/1.5 - Parametros/0 - Parametros con Python/"

Union de parametros

In [ ]:
parametro = pd.concat([dorinka,laAnonima,cencosud,inc],ignore_index=True)

Transformaciones

In [ ]:
from pandas.io.formats.format import decimal
parametro['CodigoCliente'] = '0' + parametro['CodigoCliente'].astype(str)
parametro['EAN'] = parametro['EAN'].astype(str)
parametro['CodigoArticuloCliente'] = parametro['CodigoArticuloCliente'].astype(str)
parametro['Jerarquia'] = parametro['Jerarquia'].astype(str)
parametro['CodigoArticuloCliente'] = parametro['CodigoArticuloCliente'].astype(str).apply(lambda x: x.replace('.0',''))
parametro['VentaQsemanal'] = parametro['VentaQsemanal'].astype(int)
parametro['CoberturaDias'] = parametro['CoberturaDias'].astype(int)
parametro['StockTienda'] = parametro['StockTienda'].astype(int)
parametro[['Jerarquia','j']] = parametro.Jerarquia.str.split('-',expand=True)
parametro = parametro.drop(['j'], axis=1)

Exportar resultado

In [ ]:
parametro.to_excel('ImportadorParametros.xlsx', index=False)
!cp ImportadorParametros.xlsx "/content/drive/Shareddrives/Trade Mktg/Trade/1.5 - Parametros/0 - Parametros con Python/"